In [3]:
# Lets first load in the relevant yearly files
import pickle

with open('/Users/rohanramesh/Documents/SportsData/NBA/YearlyBasicData.pickle', 'rb') as handle:
    Yearly_df = pickle.load(handle)

with open('/Users/rohanramesh/Documents/SportsData/NBA/YearlyAdvData.pickle', 'rb') as handle:
    Yearly_df_adv = pickle.load(handle)

print(Yearly_df_adv['2018'])

      3PAr  AST%   Age  BLK%   BPM  DBPM  DRB%  DWS    FTr     G ...  STL%  \
0    0.759   3.4  24.0   0.6  -2.2  -1.7   8.9  1.0  0.158  75.0 ...   1.7   
1    0.800   6.0  27.0   1.6  -2.2  -0.2  17.0  1.1  0.164  70.0 ...   1.2   
2    0.003   5.5  24.0   2.8   3.3   1.1  13.9  3.0  0.403  76.0 ...   1.8   
3    0.021  11.0  20.0   2.5   0.2   1.8  21.6  1.9  0.526  69.0 ...   1.2   
4    0.432   6.2  32.0   1.1  -5.8  -1.8  10.1  0.2  0.160  53.0 ...   0.3   
5    0.000   8.2  29.0   1.8  -6.9   0.1  28.6  0.1  0.400  21.0 ...   2.0   
6    0.068  11.3  32.0   3.0   3.3   0.3  17.3  3.5  0.296  75.0 ...   0.9   
7    0.038   5.4  19.0   4.6   0.2   1.4  18.2  1.5  0.370  72.0 ...   0.9   
8    0.500  15.2  25.0   1.6  -6.4   0.3   7.1  0.1  0.409  18.0 ...   1.4   
9    0.132   4.6  36.0   0.9  -5.2  -1.3  10.4  0.2  0.231  22.0 ...   1.9   
10   0.582   5.8  27.0   1.6   1.1   1.9  22.3  3.2  0.137  69.0 ...   1.9   
11   0.510   7.1  24.0   1.1  -2.0  -1.5  13.6  0.7  0.188  38.0

In [167]:
# Lets look at pairs of years and find those positional players that had the largest jump in some statistical category
import numpy as np
import pandas as pd
curr_basic_header = list(Yearly_df['2018'])
curr_adv_header = list(Yearly_df_adv['2018'])
stat_to_consider = 'PER'
stats_include = ['Player', stat_to_consider, 'MP', 'Pos']
position_to_test = 'PF'
for curr_year in range(2018,2017,-1):
    # lets first declare a position
    idx = (Yearly_df_adv[str(curr_year)]['Pos'] == position_to_test) & (Yearly_df_adv[str(curr_year)]['MP'] > 400)
    curr_stat = Yearly_df_adv[str(curr_year)][stats_include][idx]
    curr_stat = keep_duplicate_player_most_minutes(curr_stat) # remove duplicates
    # for previous year don't care about what position was
    idx = (Yearly_df_adv[str(curr_year-1)]['MP'] > 400)
    previous_yr_stat = Yearly_df_adv[str(curr_year-1)][stats_include][idx]
    previous_yr_stat = keep_duplicate_player_most_minutes(previous_yr_stat)
    # now for each player iterate through and look the year before for that player
    print(np.shape(curr_stat))
    stat_difference = []
    for curr_player in curr_stat['Player']:
        prev_stat = previous_yr_stat[stats_include][previous_yr_stat['Player'] == curr_player]
        if prev_stat.empty:
            print(curr_player)
        else:
#             print(prev_stat)
            stat_player_curr_year = curr_stat[stat_to_consider][curr_stat['Player'] == curr_player].tolist()
            stat_player_last_year = prev_stat[stat_to_consider].tolist()
            print(stat_player_curr_year)
            print(stat_player_last_year)
            print(stat_player_curr_year[0]-stat_player_last_year[0])
#             stat_difference.append(stat_player_curr_year-prev_stat[stat_to_consider])
#     print(stat_difference)
    

(69, 4)
[8.2]
[11.8]
-3.6000000000000014
[12.0]
[11.3]
0.6999999999999993
[12.6]
[13.5]
-0.9000000000000004
[27.3]
[26.1]
1.1999999999999993
[12.7]
[17.9]
-5.199999999999999
[15.8]
[16.3]
-0.5
[17.8]
[17.8]
0.0
[7.1]
[5.3]
1.7999999999999998
[14.1]
[12.9]
1.1999999999999993
[12.6]
[11.0]
1.5999999999999996
[14.9]
[15.7]
-0.7999999999999989
Tyler Cavanaugh
[11.1]
[12.3]
-1.200000000000001
John Collins
[28.9]
[27.5]
1.3999999999999986
[11.5]
[13.1]
-1.5999999999999996
Cheick Diallo
[8.8]
[10.6]
-1.799999999999999
[26.0]
[27.6]
-1.6000000000000014
[17.4]
[20.3]
-2.900000000000002
[7.6]
[7.7]
-0.10000000000000053
[18.0]
[17.9]
0.10000000000000142
[15.4]
[15.0]
0.40000000000000036
[16.5]
[14.5]
2.0
[16.2]
[10.1]
6.1
[16.1]
[16.5]
-0.3999999999999986
[14.0]
[13.5]
0.5
[14.8]
[10.5]
4.300000000000001
[19.6]
[22.6]
-3.0
[17.1]
[16.9]
0.20000000000000284
Josh Huestis
[14.7]
[16.4]
-1.6999999999999993
[14.9]
[14.6]
0.3000000000000007
Jonathan Isaac
[28.6]
[27.0]
1.6000000000000014
[13.2]
[9.3]
3

In [50]:
print(curr_adv_header)

['3PAr', 'AST%', 'Age', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'G', 'MP', 'OBPM', 'ORB%', 'OWS', 'PER', 'Player', 'Pos', 'STL%', 'TOV%', 'TRB%', 'TS%', 'Tm', 'USG%', 'VORP', 'WS', 'WS/48', '\xa0']


In [85]:
curr_player = 'Quincy Acy'
# print(previous_yr_stat['Player'] == curr_player)
tmpR = previous_yr_stat[['Player', 'PER', 'MP']][previous_yr_stat['Player'] == curr_player]
print(np.shape(tmpR)[0])

2


In [135]:
def keep_duplicate_player_most_minutes(input_df):
    idx_to_keep = []
    for curr_player in input_df['Player']:
        check_dup = input_df[['Player', stat_to_consider, 'MP']][input_df['Player'] == curr_player]
        idx_to_keep.append(check_dup['MP'].idxmax())
    idx_to_keep = np.unique(idx_to_keep)
    output_df = input_df.loc[list(idx_to_keep)]
    return output_df

In [137]:
curr_stat = keep_duplicate_player_most_minutes(curr_stat)

In [138]:
print(curr_stat)

                    Player   PER      MP
1               Quincy Acy   8.2  1359.0
10         Al-Farouq Aminu  12.0  2072.0
13           Ryan Anderson  12.6  1725.0
15   Giannis Antetokounmpo  27.3  2756.0
16         Carmelo Anthony  12.7  2501.0
34         Harrison Barnes  15.8  2634.0
42         Michael Beasley  17.8  1653.0
48           Dragan Bender   7.1  2069.0
49           Davis Bertans  14.1  1086.0
54         Nemanja Bjelica  12.6  1371.0
66           Trevor Booker  14.9  1158.0
110        Tyler Cavanaugh  13.0   518.0
114        Marquese Chriss  11.1  1527.0
123           John Collins  18.3  1785.0
148          Anthony Davis  28.9  2727.0
152             Sam Dekker  11.5   883.0
157          Cheick Diallo  17.0   581.0
169           Jared Dudley   8.8   686.0
171           Kevin Durant  26.0  2325.0
186         Kenneth Faried  17.4   461.0
195    Dorian Finney-Smith   7.6   448.0
208               Rudy Gay  18.0  1231.0
212             Taj Gibson  15.4  2726.0
215           Aa